In [2]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests
import numpy as np
from bs4 import BeautifulSoup

dbname = "fdac18mp2" #please use this database
collname = "glprj_bbass11" #please modify so you store data in your collection
# beginning page index
begin = "0"
client = pymongo.MongoClient()
letter = "b"

db = client[dbname]
coll = db[collname]


beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"


gleft = 10

header = {'per_page': 99}

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

# send queries and extract urls 
def get(url, coll):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0
    count = 0

    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            for el in array:
                if el['name'].lower().startswith(letter):
                    el['site'] = "git"
                    count += 1
                    coll.insert_one(el)
                    if count > 49:
                        return
                
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            if el['name'].lower().startswith(letter):
                                el['site'] = "git"
                                count += 1
                                coll.insert_one(el)
                                if count > 49:
                                    return
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')

In [22]:
#start retrieving        
get(beginurl,coll)

In [11]:
proj_array = np.array([])


for proj in coll.find({}):
    if proj['name'].lower().startswith(letter):
        projs = (proj)
        proj_array = np.append(proj_array, projs)
        
print(proj_array[0])

{'last_activity_at': '2018-10-31T18:50:57.212Z', 'site': 'git', 'default_branch': None, 'name': 'basketball', 'path': 'basketball', 'name_with_namespace': 'freedumbytes / basketball', 'id': 9154235, 'web_url': 'https://gitlab.com/freedumbytes/basketball', 'ssh_url_to_repo': 'git@gitlab.com:freedumbytes/basketball.git', 'description': '', 'created_at': '2018-10-31T18:50:57.212Z', 'tag_list': [], 'avatar_url': None, 'http_url_to_repo': 'https://gitlab.com/freedumbytes/basketball.git', 'forks_count': 0, '_id': ObjectId('5bd9f9cef318b95175425a3b'), 'namespace': {'full_path': 'freedumbytes', 'name': 'freedumbytes', 'parent_id': None, 'path': 'freedumbytes', 'kind': 'group', 'id': 1067456}, 'star_count': 0, 'readme_url': None, 'path_with_namespace': 'freedumbytes/basketball'}


In [12]:
# SourceForge
Pages = 50

url = []

# Collect URLs
for i in range(1,Pages):
    url.append("https://sourceforge.net/directory/os%3Awindows/?q=b&page=" + str(i))
    
# Get URL responses
response = [requests.get(link) for link in url]

In [13]:
# Function to select only unique entries in list
def unique(seq): 
    
   projs_checked = []

   for el in seq:
       if el not in projs_checked:
           projs_checked.append(el)
   return projs_checked

In [14]:
# Iterate through responses and extract project names
b_proj=[]
for resp in response:
    
    html_soup = BeautifulSoup(resp.text, 'html.parser')
    soup_string = str(html_soup)
    
    
    regex = "(?<=\/projects\/).+?(?=\/)"
    matches = re.findall(regex, soup_string)
    b_matches = [i for i in matches if i.lower().startswith('b')]
    b_proj.append(b_matches)

In [15]:
print(b_proj)

[['bmaxwell', 'bmaxwell', 'bmaxwell', 'bmaxwell', 'bmaxwell', 'b-e-c', 'b-e-c', 'b-e-c', 'b-e-c', 'b-e-c', 'b-i-b-l-e', 'b-i-b-l-e', 'b-i-b-l-e', 'b-i-b-l-e', 'btreeindextodbf', 'btreeindextodbf', 'btreeindextodbf', 'btreeindextodbf', 'b-gaming-for-tester', 'b-gaming-for-tester', 'b-gaming-for-tester', 'b-gaming-for-tester', 'bowtie-bio', 'bowtie-bio', 'bowtie-bio', 'bowtie-bio', 'bowtie-bio'], ['basszxtune', 'basszxtune', 'basszxtune', 'basszxtune'], ['b-basic', 'b-basic', 'b-basic', 'b-basic'], [], ['bcpl', 'bcpl', 'bcpl', 'bcpl'], ['bincmp', 'bincmp', 'bincmp', 'bincmp', 'b-ulysse-cc', 'b-ulysse-cc', 'b-ulysse-cc', 'b-ulysse-cc', 'benchmarksql2', 'benchmarksql2', 'benchmarksql2', 'benchmarksql2', 'bossa-fork', 'bossa-fork', 'bossa-fork', 'bossa-fork'], ['bliteos', 'bliteos', 'bliteos', 'bliteos', 'bgaming', 'bgaming', 'bgaming', 'bgaming'], ['bartrefiner', 'bartrefiner', 'bartrefiner', 'bartrefiner', 'b-linux', 'b-linux', 'b-linux', 'b-linux', 'bcomp', 'bcomp', 'bcomp', 'bcomp', 'bl

In [16]:
# Make list of unique project names
b_list = unique([item for sublist in b_proj for item in sublist])

print(len(b_list))

88


In [17]:
# Iterate through projects
sf_api= "https://sourceforge.net/rest/p/"

projects = []
proj = []
for proj in b_list:
    
    resp = requests.get(sf_api + proj)
    if(resp.status_code == 404):
        continue
        
    text = json.loads(resp.text)
    if(text['status'] == 'active'):
        projects.append(text)

In [18]:
print(projects[0])

{'screenshots': [{'url': 'https://sourceforge.net/p/bmaxwell/screenshot/bm1.jpg', 'caption': '3. Physical Sky Settings', 'thumbnail_url': 'https://sourceforge.net/p/bmaxwell/screenshot/bm1.jpg/thumb'}, {'url': 'https://sourceforge.net/p/bmaxwell/screenshot/maxwell_test.jpg', 'caption': '5. Rendered Image', 'thumbnail_url': 'https://sourceforge.net/p/bmaxwell/screenshot/maxwell_test.jpg/thumb'}, {'url': 'https://sourceforge.net/p/bmaxwell/screenshot/bm2.jpg', 'caption': '4. Imported Scene (Dupli Verts and Faces as Instances)', 'thumbnail_url': 'https://sourceforge.net/p/bmaxwell/screenshot/bm2.jpg/thumb'}, {'url': 'https://sourceforge.net/p/bmaxwell/screenshot/hair.jpg', 'caption': '2. Simple image fully prepared for rendering in Blender', 'thumbnail_url': 'https://sourceforge.net/p/bmaxwell/screenshot/hair.jpg/thumb'}, {'url': 'https://sourceforge.net/p/bmaxwell/screenshot/9-hair-uv-monkey1.jpg', 'caption': '1. Textured fur', 'thumbnail_url': 'https://sourceforge.net/p/bmaxwell/screens

In [19]:
count = 0

for el in projects:
        el['site'] = "SF"
        count += 1
        coll.insert_one(el)
        if count > 49:
            break

In [3]:
proj_array = np.array([])

for proj in coll.find({}):
    if proj['site'] == 'SF':
        projs = (proj)
        proj_array = np.append(proj_array, projs)
        
print(len(proj_array))

50


In [21]:
# Ooops stored too much stuff :(
print(proj_array[3000])

IndexError: index 3000 is out of bounds for axis 0 with size 91